In [2]:
%load_ext autoreload
%autoreload 2

from IPython.display import Image
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import os
import json
import numpy as np
import jax
import pickle
import matplotlib.pyplot as plt
import pandas as pd
from timecast.learners import AR
from timecast.learners._ar import _ar_predict, _ar_batch_window
from timecast.utils.numpy import ecdf
from timecast.utils.losses import MeanSquareError
import torch
import matplotlib

plt.rcParams['figure.figsize'] = [20, 10]

import tqdm.notebook as tqdm

/home/dsuo/miniconda3/envs/toy_flood/lib/python3.7/site-packages/jax/lib/xla_bridge.py:123: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [31]:
results = {}
for i in tqdm.tqdm([1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]):
    results[i] = {}
    for j in np.linspace(-8, 0.75, 36):
        if j == -8 or j == 0:
            name = "{}_{}".format(i, int(j))
        else:
            name = ".".join("{0}_{1:.2f}".format(i, j).split("0."))
        with open("../tmp/batch_{}.pkl".format(name), "rb") as f:
            results[i][j] = pickle.load(f)

In [27]:
data = {}
for batch_key, batch in results.items():
    data[batch_key] = {}
    for lr_key, lr in batch.items():
        key = list(lr.keys())[-1]
        data[batch_key][lr_key] = lr[key]["avg_mse"]

In [28]:
data = pd.DataFrame.from_dict(data)

In [29]:
data

,1,2,4,8,16,32,64,128,256,512,1024
-8.00,3.2233737,3.2233737,3.2233737,3.2233737,3.2233737,3.2233737,3.2233737,3.2233737,3.2233737,3.2233737,3.2233737
-7.75,3.221267,3.221267,3.221267,3.221267,3.221267,3.221267,3.221267,3.221267,3.221267,3.221267,3.221267
-7.50,3.2183166,3.2183166,3.2183166,3.2183166,3.2183166,3.2183166,3.2183166,3.2183166,3.2183166,3.2183166,3.2183166
-7.25,3.2144902,3.2144902,3.2144902,3.2144902,3.2144902,3.2144902,3.2144902,3.2144902,3.2144902,3.2144902,3.2144902
-7.00,3.2095773,3.2095773,3.2095773,3.2095773,3.2095773,3.2095773,3.2095773,3.2095773,3.2095773,3.2095773,3.2095773
-6.75,3.2029912,3.2029912,3.2029912,3.2029912,3.2029912,3.2029912,3.2029912,3.2029912,3.2029912,3.2029912,3.2029912
-6.50,3.1939757,3.1939757,3.1939757,3.1939757,3.1939757,3.1939757,3.1939757,3.1939757,3.1939757,3.1939757,3.1939757
-6.25,3.1815355,3.1815355,3.1815355,3.1815355,3.1815355,3.1815355,3.1815355,3.1815355,3.1815355,3.1815355,3.1815355
-6.00,3.1640656,3.1640656,3.1640656,3.1640656,3.1640656,3.1640656,3.1640656,3.1640656,3.1640656,3.1640656,3.1640656
-5.75,3.1404521,3.1404521,3.1404521,3.1404521,3.1404521,3.1404521,3.1404521,3.1404521,3.1404521,3.1404521,3.1404521


In [30]:
results[1024][-5]

{'01022500': {'mse': array(0.67312366, dtype=float32),
  'nse': array(0.863283, dtype=float32),
  'count': 3652,
  'avg_mse': array(nan, dtype=float32),
  'avg_nse': array(nan, dtype=float32)},
 '01031500': {'mse': array(1.1120733, dtype=float32),
  'nse': array(0.88161886, dtype=float32),
  'count': 3652,
  'avg_mse': array(0.67312366, dtype=float32),
  'avg_nse': array(0.863283, dtype=float32)},
 '01047000': {'mse': array(1.7105367, dtype=float32),
  'nse': array(0.8397206, dtype=float32),
  'count': 3652,
  'avg_mse': array(0.8925985, dtype=float32),
  'avg_nse': array(0.87245095, dtype=float32)},
 '01052500': {'mse': array(2.045531, dtype=float32),
  'nse': array(0.8438084, dtype=float32),
  'count': 3652,
  'avg_mse': array(1.1652446, dtype=float32),
  'avg_nse': array(0.8615408, dtype=float32)},
 '01054200': {'mse': array(7.722836, dtype=float32),
  'nse': array(0.737964, dtype=float32),
  'count': 3652,
  'avg_mse': array(1.3853161, dtype=float32),
  'avg_nse': array(0.8571077, 